## Notebook 02: Feature Engineering

**Objective:**  
Transform the cleaned flight dataset into a stable, leakage-free feature set suitable for predictive modeling.  
Features are selected based on domain relevance, availability at booking time, and modeling cost–benefit trade-offs.

**Input:**  
- Cleaned dataset from Notebook 01

**Output:**  
- Final feature matrix (X)  
- Target variable (y)  
- Saved feature dataset for modeling notebooks


In [361]:
import pandas as pd
import numpy as np

## Load cleaned data and convert data types

In [362]:
df = pd.read_csv('../Data/Cleaned/cleaned_flight_data.csv')

date_cols = ["departure_date", "booking_date"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col])

## Target variable transformation (Price)
Price is log-transformed to reduce right skew and stabilize variance.  
Model predictions are evaluated in log space and converted back for business interpretation.


In [363]:
df['target_price'] = np.log1p(df["price"])
df['target_price']

0       6.384823
1       6.687383
2       5.032005
3       6.336312
4       6.401420
          ...   
1804    4.834773
1805    6.503210
1806    6.157720
1807    6.476895
1808    6.708707
Name: target_price, Length: 1809, dtype: float64

## Booking and Temporal Features

### Days To Departure

Days-to-departure was represented both as a continuous variable and as booking-window bins.

They are binned by time periods i.e 1week, 2 week, 1 month, 3 months, 6 months. Price does not scale linearly with time and hence it makes more sense to bin the days before departure into bins of different time frames.

Binning reflects known airline pricing regimes and improves interpretability for linear models
and downstream decision logic.


In [364]:
df["days_to_departure"] = (
    pd.to_datetime(df["departure_date"]) -
    pd.to_datetime(df["booking_date"])
).dt.days

df["days_to_departure"] = df["days_to_departure"].clip(0, 365)

df["days_to_departure_bin"] = pd.cut(
    df["days_to_departure"],
    bins=[0, 7, 14, 30, 90, 180, 365],
    labels=False
)

## Calendar-Based Feature

### Deparutre Day and Weekend

Since weekend vs weekdays give a better indicator than individual days, only weekend indicator is used per model. Using binary encoding (0 for weekday and 1 for weekend)

In [365]:
df['departure_day'] = df['departure_date'].dt.day_name()
df["departure_month"] = df["departure_date"].dt.month
df["is_weekend"] = df["departure_day"].isin([5, 6]).astype(int)

### Month and Season

Month and season both represent annual demand cycles at different granularities.
To avoid redundancy, only one representation is used per model. Since season is a categorical feature, one-hot encoding is used.


In [366]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['departure_date'].dt.month.apply(get_season)
df["season"] = df["season"].astype("category")


Season was encoded using one-hot encoding to represent annual demand cycles without
imposing artificial ordinal relationships between seasons. Fall was used as the
reference category.


In [367]:

df = pd.get_dummies(
    df,
    columns=["season"],
    drop_first=True
)

df.head()

,booking_date,departure_date,origin,destination,airline,price,stops,target_price,days_to_departure,days_to_departure_bin,departure_day,departure_month,is_weekend,season_Spring,season_Summer,season_Winter
0,2023-09-20,2024-01-01,JFK,LAX,Delta,591.78,0,6.384823,103,4,Monday,1,0,False,False,True
1,2023-12-11,2024-01-01,JFK,LAX,United,801.22,0,6.687383,21,2,Monday,1,0,False,False,True
2,2023-10-18,2024-01-01,LHR,CDG,Ryanair,152.24,0,5.032005,75,3,Monday,1,0,False,False,True
3,2023-12-30,2024-01-01,SYD,MEL,Qantas,563.71,0,6.336312,2,0,Monday,1,0,False,False,True
4,2023-12-12,2024-01-02,JFK,LAX,Delta,601.70,0,6.401420,21,2,Tuesday,1,0,False,False,True


In [368]:
country_holidays = pd.read_csv("../Data/Raw/PublicHolidaysfor2024.csv")
country_holidays['date'] = pd.to_datetime(country_holidays['date'])
df['is_holiday'] = df['departure_date'].isin(country_holidays['date']).astype(int)

In [369]:
df.head()

,booking_date,departure_date,origin,destination,airline,price,stops,target_price,days_to_departure,days_to_departure_bin,departure_day,departure_month,is_weekend,season_Spring,season_Summer,season_Winter,is_holiday
0,2023-09-20,2024-01-01,JFK,LAX,Delta,591.78,0,6.384823,103,4,Monday,1,0,False,False,True,1
1,2023-12-11,2024-01-01,JFK,LAX,United,801.22,0,6.687383,21,2,Monday,1,0,False,False,True,1
2,2023-10-18,2024-01-01,LHR,CDG,Ryanair,152.24,0,5.032005,75,3,Monday,1,0,False,False,True,1
3,2023-12-30,2024-01-01,SYD,MEL,Qantas,563.71,0,6.336312,2,0,Monday,1,0,False,False,True,1
4,2023-12-12,2024-01-02,JFK,LAX,Delta,601.70,0,6.401420,21,2,Tuesday,1,0,False,False,True,0


## Catergorical Feature Encoding

### Route

Route represents the origin–destination city pair and is a nominal categorical feature.
In this dataset, only four unique routes are present, resulting in low cardinality.

Given the limited number of routes, route was encoded directly using one-hot encoding
without additional grouping. One route category was dropped to act as the reference
category, enabling the model to learn relative price differences across routes.


In [370]:
df['route'] = df['origin'] +'-'+ df['destination']
df = pd.get_dummies(
    df,
    columns=["route"],
    drop_first=True
)
df.dtypes

booking_date             datetime64[ns]
departure_date           datetime64[ns]
origin                           object
destination                      object
airline                          object
price                           float64
stops                             int64
target_price                    float64
days_to_departure                 int64
days_to_departure_bin             int64
departure_day                    object
departure_month                   int32
is_weekend                        int64
season_Spring                      bool
season_Summer                      bool
season_Winter                      bool
is_holiday                        int64
route_JFK-LAX                      bool
route_LHR-CDG                      bool
route_SYD-MEL                      bool
dtype: object

### Airline

Airline is a nominal categorical feature with low cardinality and no inherent ordering.
Different airlines apply distinct pricing strategies, service levels, and cost structures,
which can independently influence ticket prices.

To capture these effects without imposing artificial ordinal relationships, airline was
encoded using one-hot encoding. One category was dropped to serve as the reference level
and to avoid multicollinearity in linear models.


In [371]:
df["airline"] = df["airline"].str.upper().str.strip()

df = pd.get_dummies(
    df,
    columns=["airline"],
    drop_first=True
)
df.head()

,booking_date,departure_date,origin,destination,price,stops,target_price,days_to_departure,days_to_departure_bin,departure_day,...,season_Summer,season_Winter,is_holiday,route_JFK-LAX,route_LHR-CDG,route_SYD-MEL,airline_INDIGO,airline_QANTAS,airline_RYANAIR,airline_UNITED
0,2023-09-20,2024-01-01,JFK,LAX,591.78,0,6.384823,103,4,Monday,...,False,True,1,True,False,False,False,False,False,False
1,2023-12-11,2024-01-01,JFK,LAX,801.22,0,6.687383,21,2,Monday,...,False,True,1,True,False,False,False,False,False,True
2,2023-10-18,2024-01-01,LHR,CDG,152.24,0,5.032005,75,3,Monday,...,False,True,1,False,True,False,False,False,True,False
3,2023-12-30,2024-01-01,SYD,MEL,563.71,0,6.336312,2,0,Monday,...,False,True,1,False,False,True,False,True,False,False
4,2023-12-12,2024-01-02,JFK,LAX,601.70,0,6.401420,21,2,Tuesday,...,False,True,0,True,False,False,False,False,False,False


## Feature Cleanup

In [372]:
delete_col = ['booking_date', 'departure_date', 'departure_day', 'origin', 'destination' , 'days_to_departure']
df = df.drop(columns = delete_col)

dummy_cols = df.select_dtypes(include="bool").columns
df[dummy_cols] = df[dummy_cols].astype(int)

df.head()

,price,stops,target_price,days_to_departure_bin,departure_month,is_weekend,season_Spring,season_Summer,season_Winter,is_holiday,route_JFK-LAX,route_LHR-CDG,route_SYD-MEL,airline_INDIGO,airline_QANTAS,airline_RYANAIR,airline_UNITED
0,591.78,0,6.384823,4,1,0,0,0,1,1,1,0,0,0,0,0,0
1,801.22,0,6.687383,2,1,0,0,0,1,1,1,0,0,0,0,0,1
2,152.24,0,5.032005,3,1,0,0,0,1,1,0,1,0,0,0,1,0
3,563.71,0,6.336312,0,1,0,0,0,1,1,0,0,1,0,1,0,0
4,601.70,0,6.401420,2,1,0,0,0,1,0,1,0,0,0,0,0,0


In [373]:
df.dtypes

price                    float64
stops                      int64
target_price             float64
days_to_departure_bin      int64
departure_month            int32
is_weekend                 int64
season_Spring              int64
season_Summer              int64
season_Winter              int64
is_holiday                 int64
route_JFK-LAX              int64
route_LHR-CDG              int64
route_SYD-MEL              int64
airline_INDIGO             int64
airline_QANTAS             int64
airline_RYANAIR            int64
airline_UNITED             int64
dtype: object

## Save Features to CSV

In [374]:
df.to_csv('../Data/Cleaned/flight_features.csv', index = False)